In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import re
import unicodedata
import nltk
from nltk.corpus import stopwords
import keras
from tqdm import tqdm
import pickle
from keras.models import Model
import keras.backend as K
from sklearn.metrics import confusion_matrix,f1_score,classification_report
import matplotlib.pyplot as plt
from keras.callbacks import ModelCheckpoint
import itertools
from keras.models import load_model
from sklearn.utils import shuffle
from transformers import *
from transformers import BertTokenizer, TFBertModel, BertConfig


In [3]:
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def clean_stopwords_shortwords(w):
    stopwords_list=stopwords.words('english')
    words = w.split() 
    clean_words = [word for word in words if (word not in stopwords_list) and len(word) > 2]
    return " ".join(clean_words) 

def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())
    w = re.sub(r"([?.!,¿])", r" ", w)
    w = re.sub(r'[" "]+', " ", w)
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
    w=clean_stopwords_shortwords(w)
    w=re.sub(r'@\w+', '',w)
    return w

In [4]:
#import data
pharm_data = pd.read_csv('pharmacy_dataset_reduced.csv')
data = pharm_data.copy()
data.head()
#len(data)

,Website,Body,Header,Footer,Image Urls,Accredited,Complete_html,Zipcode,Valid_phone
0,https://acariahealth.envolvehealth.com,Skip to Main Content Home Contact Insights Sea...,Home Contact Insights Search Search About Over...,Locations Referral Forms Careers Disclaimer HI...,['https://acariahealth.envolvehealth.com/conte...,0,Skip to Main Content Home Contact Insights Sea...,0,1
1,https://alto.com,Shop Alto Essentials to add pharmacy products ...,Shop Alto Essentials to add pharmacy products ...,Our Story Careers Drive for Alto For Providers...,['https://images.prismic.io/alto/176341aa-258d...,0,Shop Alto Essentials to add pharmacy products ...,0,1
2,https://www.amberpharmacy.com,Loading... Skip to navigation Skip to main con...,NaN,About Careers 340B Hospital Program Locations ...,['https://www.amberpharmacy.com/wp-content/the...,0,NaN,0,0
3,https://www.aoncology.com,For Patients Careers Blog Practice Success Pat...,For Patients Careers Blog Practice Success Pat...,"© 2020 American Oncology Network, LLC. All Rig...",['https://dc.ads.linkedin.com/collect/?pid=944...,0,For Patients Careers Blog Practice Success Pat...,1,0
4,https://www.birdirx.com,Loading... × Close alert Fly with Birdi! Free...,NaN,Support Hours of Operation Monday through Frid...,"[""https://www.birdirx.comdata:image/svg+xml;ch...",0,NaN,0,0


### Data Cleaning

In [5]:
#combine columns into new column containing all text
data['text'] = data['Body'].astype(str) + data['Header'].astype(str) + data['Footer'].astype(str)
data.head()

,Website,Body,Header,Footer,Image Urls,Accredited,Complete_html,Zipcode,Valid_phone,text
0,https://acariahealth.envolvehealth.com,Skip to Main Content Home Contact Insights Sea...,Home Contact Insights Search Search About Over...,Locations Referral Forms Careers Disclaimer HI...,['https://acariahealth.envolvehealth.com/conte...,0,Skip to Main Content Home Contact Insights Sea...,0,1,Skip to Main Content Home Contact Insights Sea...
1,https://alto.com,Shop Alto Essentials to add pharmacy products ...,Shop Alto Essentials to add pharmacy products ...,Our Story Careers Drive for Alto For Providers...,['https://images.prismic.io/alto/176341aa-258d...,0,Shop Alto Essentials to add pharmacy products ...,0,1,Shop Alto Essentials to add pharmacy products ...
2,https://www.amberpharmacy.com,Loading... Skip to navigation Skip to main con...,NaN,About Careers 340B Hospital Program Locations ...,['https://www.amberpharmacy.com/wp-content/the...,0,NaN,0,0,Loading... Skip to navigation Skip to main con...
3,https://www.aoncology.com,For Patients Careers Blog Practice Success Pat...,For Patients Careers Blog Practice Success Pat...,"© 2020 American Oncology Network, LLC. All Rig...",['https://dc.ads.linkedin.com/collect/?pid=944...,0,For Patients Careers Blog Practice Success Pat...,1,0,For Patients Careers Blog Practice Success Pat...
4,https://www.birdirx.com,Loading... × Close alert Fly with Birdi! Free...,NaN,Support Hours of Operation Monday through Frid...,"[""https://www.birdirx.comdata:image/svg+xml;ch...",0,NaN,0,0,Loading... × Close alert Fly with Birdi! Free...


In [6]:
#rename Accredited as 'label'
data=data.rename(columns = {'Accredited': 'label'}, inplace = False)
data.head()

,Website,Body,Header,Footer,Image Urls,label,Complete_html,Zipcode,Valid_phone,text
0,https://acariahealth.envolvehealth.com,Skip to Main Content Home Contact Insights Sea...,Home Contact Insights Search Search About Over...,Locations Referral Forms Careers Disclaimer HI...,['https://acariahealth.envolvehealth.com/conte...,0,Skip to Main Content Home Contact Insights Sea...,0,1,Skip to Main Content Home Contact Insights Sea...
1,https://alto.com,Shop Alto Essentials to add pharmacy products ...,Shop Alto Essentials to add pharmacy products ...,Our Story Careers Drive for Alto For Providers...,['https://images.prismic.io/alto/176341aa-258d...,0,Shop Alto Essentials to add pharmacy products ...,0,1,Shop Alto Essentials to add pharmacy products ...
2,https://www.amberpharmacy.com,Loading... Skip to navigation Skip to main con...,NaN,About Careers 340B Hospital Program Locations ...,['https://www.amberpharmacy.com/wp-content/the...,0,NaN,0,0,Loading... Skip to navigation Skip to main con...
3,https://www.aoncology.com,For Patients Careers Blog Practice Success Pat...,For Patients Careers Blog Practice Success Pat...,"© 2020 American Oncology Network, LLC. All Rig...",['https://dc.ads.linkedin.com/collect/?pid=944...,0,For Patients Careers Blog Practice Success Pat...,1,0,For Patients Careers Blog Practice Success Pat...
4,https://www.birdirx.com,Loading... × Close alert Fly with Birdi! Free...,NaN,Support Hours of Operation Monday through Frid...,"[""https://www.birdirx.comdata:image/svg+xml;ch...",0,NaN,0,0,Loading... × Close alert Fly with Birdi! Free...


In [7]:
#reduce dataset to only the columns we need, 'label' and 'text'
data = data.drop(['Website', 'Body', 'Header', 'Footer', 'Image Urls', 'Complete_html', 'Zipcode', 'Valid_phone'], axis=1)

In [8]:
#Sentences contain the entire text data and labels contain all the corresponding labels
data.head()

,label,text
0,0,Skip to Main Content Home Contact Insights Sea...
1,0,Shop Alto Essentials to add pharmacy products ...
2,0,Loading... Skip to navigation Skip to main con...
3,0,For Patients Careers Blog Practice Success Pat...
4,0,Loading... × Close alert Fly with Birdi! Free...


In [9]:
len(data)

72

In [10]:
#drop any missing values
data = data.dropna()

In [11]:
len(data)

72

In [12]:
#reset index
data=data.reset_index(drop=True) 
#shuffle data
data = shuffle(data)   
data.head()

,label,text
8,0,Atrium Health | CarolinaCARE Welcome to Caroli...
38,1,"Hello, Guest Blog Reviews Payment Methods Freq..."
7,0,Skip to main content Skip to footer content Mo...
19,0,[email protected] 1-866-422-4866 ORDER REFILLS...
53,1,Welcome to buy-online-antibiotics.com To chang...


In [13]:
#apply the function to clean the text - all lower case, strip spaces and characters 
data['text']=data['text'].map(preprocess_sentence)    
data.head()

,label,text
8,0,atrium health carolinacare welcome carolinacar...
38,1,hello guest blog reviews payment methods frequ...
7,0,skip main content skip footer content humana p...
19,0,email protected order refills customer login c...
53,1,welcome buy online antibiotics com change page...


### Setting up a pre-trained BERT model for fine-tuning
#### Load BERT tokenizer and BERT model

In [14]:
#define the number of unique classes under 'label'
num_classes=len(data.label.unique())
num_classes

2

In [15]:
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', 
                                                             num_labels=num_classes)

loading file vocab.txt from cache at C:\Users\haley/.cache\huggingface\hub\models--bert-base-uncased\snapshots\5546055f03398095e385d7dc625e636cc8910bf2\vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at C:\Users\haley/.cache\huggingface\hub\models--bert-base-uncased\snapshots\5546055f03398095e385d7dc625e636cc8910bf2\tokenizer_config.json
loading configuration file config.json from cache at C:\Users\haley/.cache\huggingface\hub\models--bert-base-uncased\snapshots\5546055f03398095e385d7dc625e636cc8910bf2\config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_

In [16]:
test_sent = "this is a test for our model."
tokenized_sequence= bert_tokenizer.encode_plus(test_sent,
                                               add_special_tokens = True, 
                                               pad_to_max_length = True, 
                                               return_attention_mask = True)

C:\Users\haley\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2304: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [17]:
#tokenizer returns a dictionary with all the arguments necessary for its corresponding model to work properly
tokenized_sequence

{'input_ids': [101, 2023, 2003, 1037, 3231, 2005, 2256, 2944, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [18]:
#Decode sentence
bert_tokenizer.decode(tokenized_sequence['input_ids'])

'[CLS] this is a test for our model. [SEP]'

In [19]:
#turn text and label column into variabels for tokenizer 
text = data['text']
labels = data['label']
len(text), len(labels) #make sure len is the same

(72, 72)

In [20]:
labels.dtype

dtype('int64')

In [21]:
text.dtype

dtype('O')

In [22]:
text = text.astype(str)

In [23]:
text.head()

8     atrium health carolinacare welcome carolinacar...
38    hello guest blog reviews payment methods frequ...
7     skip main content skip footer content humana p...
19    email protected order refills customer login c...
53    welcome buy online antibiotics com change page...
Name: text, dtype: object

#### Encoding of the text data using BERT Tokenizer and obtaining the input_ids and attentions masks to feed into the model

In [24]:
#Load the sentences into the BERT Tokenizer
#BERT Tokenizer returns a dictionary - get input ds and the attention masks
#Convert encoding to NumPy arrays
'''
input_ids = []
attention_masks = []

for sent in text:
    #print(sent)
    bert_inp = bert_tokenizer.encode_plus(sent, return_attention_mask = True)
    #print(bert_inp)
    input_ids.append(bert_inp['input_ids'])
    attention_masks.append(bert_inp['attention_mask'])
    #break
#input_ids.astype('object')
input_ids = np.array(input_ids, dtype=object)
attention_masks = np.array(attention_masks, dtype=object)
labels = np.array(labels)
'''

"\ninput_ids = []\nattention_masks = []\n\nfor sent in text:\n    #print(sent)\n    bert_inp = bert_tokenizer.encode_plus(sent, return_attention_mask = True)\n    #print(bert_inp)\n    input_ids.append(bert_inp['input_ids'])\n    attention_masks.append(bert_inp['attention_mask'])\n    #break\n#input_ids.astype('object')\ninput_ids = np.array(input_ids, dtype=object)\nattention_masks = np.array(attention_masks, dtype=object)\nlabels = np.array(labels)\n"

In [25]:
input_ids.dtype

NameError: name 'input_ids' is not defined

In [ ]:
len(input_ids), len(attention_masks), len(labels) #make sure they're all the same length

### Split Data - Training & Validation
- (80-20 split)

In [ ]:
#Split into the random train and validation subsets
'''
train_inp, val_inp, train_label, val_label, train_mask, val_mask = train_test_split(input_ids,
                                                                                    labels,
                                                                                    attention_masks,
                                                                                    test_size=0.2)

len(train_inp), len(val_inp), len(train_label), len(val_label), len(train_mask), len(val_mask)'''

In [ ]:
#X = data['text']
#y = data['label']

In [ ]:
#Split into the random train and validation subsets
#X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
#len(X_train), len(X_val), len(y_train), len(y_val)

In [ ]:
y = tf.keras.utils.to_categorical(data["label"].values, num_classes=num_classes)
#y

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data['text'], y, test_size=0.25)

In [ ]:
len(x_train), len(x_test), len(y_train), len(y_test)

In [ ]:
#train_dataset = glue_convert_examples_to_features(data['text'], bert_tokenizer, max_length=128, task='mrpc')

In [26]:
import tensorflow_hub as hub
import tensorflow_text as text

preprocessor = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-preprocess/2")
encoder = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-base/1")


def get_embeddings(sentences):
  '''return BERT-like embeddings of input text
  Args:
    - sentences: list of strings
  Output:
    - BERT-like embeddings: tf.Tensor of shape=(len(sentences), 768)
  '''
  preprocessed_text = preprocessor(sentences)
  return encoder(preprocessed_text)['pooled_output']


get_embeddings([
    "Questa collezione di Haiku è una porta aperta sulla cultura giapponese."]
)

ModuleNotFoundError: No module named 'tensorflow_text'

### Loss, Metric, Optmizer

In [ ]:
#a 'callback' can perform actions at various stages of training - write logs, save model to disk
    #early stopping, view of internal state/stats during training
'''
log_dir='tensorboard_data/tb_bert'
model_save_path='./models/bert_model.h5'

callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,
                                                save_weights_only=True,
                                                monitor='val_loss',
                                                mode='min',
                                                save_best_only=True),
             keras.callbacks.TensorBoard(log_dir=log_dir)]

print('\nBert Model', bert_model.summary())
'''
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5, epsilon=1e-08)

bert_model.compile(loss=loss, optimizer=optimizer, metrics=[metric])

In [ ]:
earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor = "val_loss", 
                                                      patience = 3,
                                                      restore_best_weights = True)

### Train Model


In [ ]:
#valid_dataset = ([val_inp, val_mask], val_label)

In [ ]:
train_model = bert_model.fit(x_train,
                             y_train,
                             batch_size=32,
                             epochs=4,
                             validation_data=(x_test, y_test),
                             callbacks=[earlystop_callback])